In [1]:
@file:DependsOn("org.jetbrains.kotlinx:kotlin-deeplearning-api:0.3.0")

import org.jetbrains.kotlinx.dl.api.core.Sequential
import org.jetbrains.kotlinx.dl.api.core.activation.Activations
import org.jetbrains.kotlinx.dl.api.core.initializer.GlorotNormal
import org.jetbrains.kotlinx.dl.api.core.initializer.Zeros
import org.jetbrains.kotlinx.dl.api.core.layer.core.Dense
import org.jetbrains.kotlinx.dl.api.core.layer.core.Input
import org.jetbrains.kotlinx.dl.api.core.loss.Losses
import org.jetbrains.kotlinx.dl.api.core.metric.Metrics
import org.jetbrains.kotlinx.dl.api.core.optimizer.Adam
import org.jetbrains.kotlinx.dl.api.core.summary.logSummary
import org.jetbrains.kotlinx.dl.dataset.OnHeapDataset
import kotlin.random.Random

In [50]:
val SEED = 12L
val TEST_BATCH_SIZE = 1000
val EPOCHS = 100
val TRAINING_BATCH_SIZE = 100

In [51]:
val model = Sequential.of(
    Input(4),
    Dense(1, Activations.Linear, kernelInitializer = GlorotNormal(), biasInitializer = Zeros(), name = "dense_2")
)

This example shows how to do regression from scratch, starting from generated dataset, using simple Dense-based [model] with 1 neuron.
It includes:
 - dataset creation
 - dataset splitting
 - model compilation
 - model training
 - model evaluation
 - model weights printing

In [52]:
val rnd = Random(SEED)

In [53]:
val data = Array(10000) { doubleArrayOf(0.0, 0.0, 0.0, 0.0, 0.0) }

for (i in data.indices) {
    data[i][1] = 2 * (rnd.nextDouble() - 0.5)
    data[i][2] = 2 * (rnd.nextDouble() - 0.5)
    data[i][3] = 2 * (rnd.nextDouble() - 0.5)
    data[i][4] = 2 * (rnd.nextDouble() - 0.5)
    data[i][0] = data[i][1] - 2 * data[i][2] + 1.5 * data[i][3] - 0.95 * data[i][4] + rnd.nextDouble(0.1)
    // 1 * x1 - 2 * x2 + 1.5 * x3  - 0.95 * x4 +- 0.1
}

data.shuffle()

In [54]:
fun extractX(): Array<FloatArray> {
    val init: (index: Int) -> FloatArray = { index ->
        floatArrayOf(
            data[index][1].toFloat(),
            data[index][2].toFloat(),
            data[index][3].toFloat(),
            data[index][4].toFloat()
        )
    }
    return Array(data.size, init = init)
}

In [55]:
fun extractY(): FloatArray {
    val labels = FloatArray(data.size) { 0.0f }
        for (i in labels.indices) {
        labels[i] = data[i][0].toFloat()
    }
    return labels
}

In [56]:
val dataset = OnHeapDataset.create(
    ::extractX,
    ::extractY
)

In [57]:
val (train, test) = dataset.split(0.9)

In [58]:
model.compile(
    optimizer = Adam(),
    loss = Losses.MSE,
    metric = Metrics.MAE
)

In [59]:
// temporary not support, https://github.com/JetBrains/KotlinDL/pull/251
model.logSummary()

In [60]:
model.fit(dataset = train, epochs = EPOCHS, batchSize = TRAINING_BATCH_SIZE)

org.jetbrains.kotlinx.dl.api.core.history.TrainingHistory@3eba57a7

In [61]:
repeat(100) { id ->
    val xReal = test.getX(id)
    val yReal = test.getY(id)
    val yPred = model.predictSoftly(xReal)

    println("xReal: ${arrayOf(xReal).contentDeepToString()}, yReal: $yReal, yPred: ${yPred[0]}")
}

xReal: [[0.6087428, -0.97591543, -0.51635, -0.02718567]], yReal: 1.9011898, yPred: 1.7880858
xReal: [[0.6506943, -0.74951607, 0.62118393, -0.5789276]], yReal: 3.640614, yPred: 3.6217148
xReal: [[0.7368443, 0.20454682, 0.5185782, -0.6787931]], yReal: 1.8144127, yPred: 1.8116987
xReal: [[-0.19058122, 0.20471321, 0.29105935, 0.24365748]], yReal: -0.3682384, yPred: -0.330162
xReal: [[0.67219007, -0.25889054, 0.8821758, 0.9686212]], yReal: 1.687433, yPred: 1.6211332
xReal: [[0.080434956, -0.37199944, 0.2912554, 0.95787024]], yReal: 0.40611953, yPred: 0.3733495
xReal: [[-0.50868046, 0.27200517, -0.62984765, 0.44977978]], yReal: -2.3983517, yPred: -2.3523724
xReal: [[0.9190222, -0.4844544, -0.48235786, 0.22850004]], yReal: 1.0334556, yPred: 0.9597334
xReal: [[-0.77795845, -0.94855833, 0.5636188, 0.5017549]], yReal: 1.4904497, yPred: 1.4677492
xReal: [[0.8380945, 0.34323326, -0.86824536, 0.4596334]], yReal: -1.578876, yPred: -1.5121273
xReal: [[0.20092645, 0.9538387, -0.29171363, -0.650504]], 

In [62]:
val mae = model.evaluate(dataset = test, batchSize = TEST_BATCH_SIZE).metrics[Metrics.MAE]
println("Weights: " + model.getLayer("dense_2").weights["dense_2_dense_kernel"].contentDeepToString())
println("Bias: " + model.getLayer("dense_2").weights["dense_2_dense_bias"].contentDeepToString())
println("MAE: $mae")

Weights: [[0.99786204], [-1.9254692], [1.4981416], [-0.9488229]]
Bias: [0.049320396]
MAE: 0.04350972920656204


In [63]:
repeat(100) { id ->
    val xReal = test.getX(id)
    val yReal = test.getY(id)

    val yPred = model.predictSoftly(xReal)

    println("xReal: ${xReal[0]}, yReal: $yReal, yPred: ${yPred[0]}")
}

xReal: 0.6087428, yReal: 1.9011898, yPred: 1.7880858
xReal: 0.6506943, yReal: 3.640614, yPred: 3.6217148
xReal: 0.7368443, yReal: 1.8144127, yPred: 1.8116987
xReal: -0.19058122, yReal: -0.3682384, yPred: -0.330162
xReal: 0.67219007, yReal: 1.687433, yPred: 1.6211332
xReal: 0.080434956, yReal: 0.40611953, yPred: 0.3733495
xReal: -0.50868046, yReal: -2.3983517, yPred: -2.3523724
xReal: 0.9190222, yReal: 1.0334556, yPred: 0.9597334
xReal: -0.77795845, yReal: 1.4904497, yPred: 1.4677492
xReal: 0.8380945, yReal: -1.578876, yPred: -1.5121273
xReal: 0.20092645, yReal: -1.4745728, yPred: -1.4065851
xReal: -0.14396854, yReal: 1.8080261, yPred: 1.7896273
xReal: 0.84789455, yReal: -0.75592464, yPred: -0.73575497
xReal: -0.124988504, yReal: -1.2425467, yPred: -1.1685106
xReal: 0.9571336, yReal: 0.76503456, yPred: 0.82035315
xReal: 0.39315963, yReal: 3.184623, yPred: 3.1047266
xReal: 0.25079665, yReal: -1.2914593, yPred: -1.3415972
xReal: -0.13490531, yReal: -1.0673596, yPred: -1.0855988
xReal: -0.

In [64]:
model.close()